clean:
abc: 0.9109
cnn: 0.9097

pgd20
alpha=0.1
abc: 0.3411
cnn: 0.3450

alpha=1
abc: 0.6949
cnn: 0.6882

alpha=0.15
abc: 0.7582
cnn: 0.7490

pgd50
abc: 0.7283
cnn: 0.7211

fgsm
abc: 0.3871
cnn: 0.3540


foolbox
fgsm: epsilon=0.031
abc: 0.1400
cnn: 0.2149

fgsm: epsilon=0.005
abccnn: 0.6807
abc: 0.4621
cnn: 0.4154

pgd20: alpha=0.1 epsilon=0.005
abccnn: 0.4328
abc: 0.2401
cnn: 0.0970


ABCCNN
clean    0.9363
fgsm     0.6807
pgd      0.4328

In [1]:
import torch
import torchvision
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils.tools import dotdict
from driver.driver import ABC_Driver
from torch_geometric_temporal import METRLADatasetLoader
from other_model.other_model import make_default_model
import atd2022
torch.cuda.is_available()
torch.cuda.set_device(0)

In [2]:
cifar10_args = dotdict()

cifar10_args.name = 'cifar10'
cifar10_args.train_batch_size = 128
cifar10_args.predict_batch_size = 128
cifar10_args.device = ['cuda:0']

cifar10_args.train_epochs = 250
cifar10_args.lr = 0.01
cifar10_args.criterion = 'CE'
cifar10_args.optimizer = 'AdamW'
cifar10_args.scheduler = 'OneCycle'
cifar10_args.attack = {'fgsm':(0.005,), 'pgd':(0.005,0.1,20)}

activation = 'relu'
input_channel = 3
# knpp = [32,64,96,128,160,192,224]

# cifar10_args.layers=[
#     ('cnn2d', ((input_channel, knpp[0], (3,3), 1, 1, 1, 1), 1, None, None, activation, False)),
#     ('atc2d', ((knpp[0], knpp[1], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, True)),
#     ('atc2d', ((knpp[1], knpp[2], (3,3), 1, 1, 1, knpp[0]), 1, 'first', (2,2), activation, True)),
#     ('atc2d', ((knpp[2], knpp[3], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, True)),
#     ('atc2d', ((knpp[3], knpp[4], (3,3), 1, 1, 1, knpp[0]), 1, 'first', (2,2), activation, True)),
#     ('atc2d', ((knpp[4], knpp[5], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, True)),
#     ('atc2d', ((knpp[5], knpp[6], (3,3), 1, 1, 1, knpp[0]), 1, 'first', (2,2), activation, True)),
#     ('cnn2d', ((knpp[-1], knpp[-1], (3,3), 1, 0, 1, knpp[0]), 1, None, None, activation, False)),
#     ('cnn2d', ((knpp[-1], knpp[-1], (2,2), 1, 0, 1, knpp[0]), 1, None, None, False, False)),
# #     ('adptavgpool', (1,1)),
#     ('linear', (knpp[-1], 10, (1,2,3)))\
# ]


knpp = [48,96,144,240,336,432,528,624]

cifar10_args.layers=[
    ('cnn2d', ((input_channel, knpp[0], (3,3), 1, 1, 1, 1), 1, None, None, activation, False)),
    ('atrc2d', ((knpp[0], knpp[1], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, True)),
    ('atrc2d', ((knpp[1], knpp[2], (3,3), 1, 1, 1, knpp[0]), 1, 'first', (2,2), activation, True)),
    ('atrc2d', ((knpp[2], knpp[3], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, True)),
    ('atrc2d', ((knpp[3], knpp[4], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, True)),
    ('atrc2d', ((knpp[4], knpp[5], (3,3), 1, 1, 1, knpp[0]), 1, 'first', (2,2), activation, True)),
    ('atrc2d', ((knpp[5], knpp[6], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, True)),
    ('atrc2d', ((knpp[6], knpp[7], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, True)),
    ('adptavgpool', (1,1)),
    ('linear', (knpp[-1], 10, (1,2,3)))
]


# knpp = [48,96,144,240,336,432,528]
# cifar10_args.layers=[
#     ('cnn2d', ((input_channel, knpp[0], (3,3), 1, 1, 1, 1), 1, None, None, activation, False)),
#     ('atrc2d', ((knpp[0], knpp[1], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, True)),
#     ('atrc2d', ((knpp[1], knpp[2], (3,3), 1, 1, 1, knpp[0]), 1, 'first', (2,2), activation, True)),
#     ('atrc2d', ((knpp[2], knpp[3], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, True)),
#     ('atrc2d', ((knpp[3], knpp[4], (3,3), 1, 1, 1, knpp[0]), 1, 'first', (2,2), activation, True)),
#     ('atrc2d', ((knpp[4], knpp[5], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, True)),
#     ('atrc2d', ((knpp[5], knpp[6], (3,3), 1, 1, 1, knpp[0]), 1, 'first', (2,2), activation, True)),
#     ('adptavgpool', (1,1)),
#     ('linear', (knpp[-1], 10, (1,2,3)))
# ]

In [ ]:
driver = ABC_Driver(cifar10_args, None, record_path=None, if_hash=False)
driver.train()

Use: ['cuda:0']
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
add record: 03/25/2023 16:31
epoch: 0, train_loss: 1.7716, test_metric: 0.4988, time: 593.2482035160065
epoch: 1, train_loss: 1.4845, test_metric: 0.5628, time: 591.7711906433105
epoch: 2, train_loss: 1.3432, test_metric: 0.6113, time: 591.7671165466309
epoch: 3, train_loss: 1.2384, test_metric: 0.6279, time: 591.7860770225525
epoch: 4, train_loss: 1.1637, test_metric: 0.6348, time: 591.573098897934
epoch: 5, train_loss: 1.0978, test_metric: 0.6952, time: 591.6569447517395
epoch: 6, train_loss: 1.0395, test_metric: 0.7138, time: 592.107753276825
epoch: 7, train_loss: 1.0017, test_metric: 0.7064, time: 592.1148927211761
epoch: 8, train_loss: 0.9595, test_metric: 0.7341, time: 591.9846529960632
epoch: 9, train_loss: 0.9299, test_metric: 0.7449, time: 591.9319047927856
epoch: 10, train_loss: 0.8987, test_metric: 0.745, time: 591.9915819168091
epoch: 11, train_l

In [4]:
# driver.model.load_state_dict(torch.load("save/CIFAR10_ARC_2023_03_23.pt"))

<All keys matched successfully>

In [3]:
driver = ABC_Driver(cifar10_args, None, record_path=None, if_hash=False)
driver.train()

Use: ['cuda:0']
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
add record: 03/15/2023 13:02
epoch: 0, train_loss: 1.7298, test_metric: 0.5006, time: 272.0483646392822
epoch: 1, train_loss: 1.2325, test_metric: 0.5223, time: 270.75030183792114
epoch: 2, train_loss: 0.9571, test_metric: 0.7083, time: 270.2101948261261
epoch: 3, train_loss: 0.8071, test_metric: 0.7231, time: 270.1269121170044
epoch: 4, train_loss: 0.7114, test_metric: 0.7605, time: 269.46717739105225
epoch: 5, train_loss: 0.6521, test_metric: 0.7366, time: 270.6508870124817
epoch: 6, train_loss: 0.6073, test_metric: 0.7402, time: 274.7116186618805
epoch: 7, train_loss: 0.5736, test_metric: 0.7947, time: 276.6718318462372
epoch: 8, train_loss: 0.5326, test_metric: 0.7047, time: 276.80812978744507
epoch: 9, train_loss: 0.5145, test_metric: 0.7839, time: 277.0181128978729
epoch: 10, train_loss: 0.4917, test_metric: 0.817, time: 276.4387309551239
epoch: 11, tr

epoch: 106, train_loss: 0.0358, test_metric: 0.9127, time: 523.6255731582642
epoch: 107, train_loss: 0.0397, test_metric: 0.9171, time: 411.3855745792389
epoch: 108, train_loss: 0.0395, test_metric: 0.9082, time: 300.6899709701538
epoch: 109, train_loss: 0.0369, test_metric: 0.9177, time: 300.8806264400482
epoch: 110, train_loss: 0.0382, test_metric: 0.9069, time: 298.46907663345337
epoch: 111, train_loss: 0.0379, test_metric: 0.9122, time: 299.9549720287323
epoch: 112, train_loss: 0.0383, test_metric: 0.9148, time: 299.8769471645355
epoch: 113, train_loss: 0.045, test_metric: 0.9146, time: 330.71510553359985
epoch: 114, train_loss: 0.0407, test_metric: 0.9141, time: 369.57953238487244
epoch: 115, train_loss: 0.0427, test_metric: 0.917, time: 300.8098452091217
epoch: 116, train_loss: 0.0442, test_metric: 0.9124, time: 299.5692527294159
epoch: 117, train_loss: 0.0456, test_metric: 0.9088, time: 299.353134393692
epoch: 118, train_loss: 0.0456, test_metric: 0.9199, time: 299.9144065380096

epoch: 213, train_loss: 0.0011, test_metric: 0.9372, time: 303.5401723384857
epoch: 214, train_loss: 0.0011, test_metric: 0.9385, time: 301.1206910610199
epoch: 215, train_loss: 0.0011, test_metric: 0.938, time: 303.4525423049927
epoch: 216, train_loss: 0.0011, test_metric: 0.9378, time: 303.42396783828735
epoch: 217, train_loss: 0.0011, test_metric: 0.9372, time: 303.54974150657654
epoch: 218, train_loss: 0.0011, test_metric: 0.9381, time: 303.65432953834534
epoch: 219, train_loss: 0.0011, test_metric: 0.9379, time: 303.26136112213135
epoch: 220, train_loss: 0.0011, test_metric: 0.9382, time: 303.2235770225525
epoch: 221, train_loss: 0.0013, test_metric: 0.9372, time: 301.07968950271606
epoch: 222, train_loss: 0.0012, test_metric: 0.9373, time: 303.3643968105316
epoch: 223, train_loss: 0.0011, test_metric: 0.9372, time: 303.28150963783264
epoch: 224, train_loss: 0.0011, test_metric: 0.9374, time: 303.676824092865
epoch: 225, train_loss: 0.0011, test_metric: 0.9378, time: 303.489892244

In [4]:
torch.save(driver.model.state_dict(), "save/CIFAR10_ARC_2023_03_23.pt")
# driver.model.load_state_dict(torch.load("save/CIFAR10_ABC_2023_03_07.pt"))

In [5]:
# driver.metric(test_attack=True)

In [5]:
from autoattack import AutoAttack

In [ ]:
AutoAttack()